<a href="https://colab.research.google.com/github/inderpreetsingh01/PyTorch/blob/main/VQ_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import numpy as np
import pickle

In [154]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

In [133]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [126]:
import torch 
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as T

In [8]:
%%capture
!pip install datasets
from datasets import load_dataset

In [146]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [9]:
rvl_cdip_dataset = load_dataset('rvl_cdip', streaming=True)

In [81]:
train_data = rvl_cdip_dataset['train']
val_data = rvl_cdip_dataset['validation']
test_data = rvl_cdip_dataset['test']

In [82]:
train_data.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(names=['letter', 'form', 'email', 'handwritten', 'advertisement', 'scientific report', 'scientific publication', 'specification', 'file folder', 'news article', 'budget', 'invoice', 'presentation', 'questionnaire', 'resume', 'memo'], id=None)}

In [83]:
transforms = T.Compose(
    [   T.Lambda(lambda x:np.resize(x, (224,224))),
        T.Lambda(lambda x:np.expand_dims(x, 2)),
        T.ToTensor()
    ]
)

In [128]:
data = list(train_data.take(1))

In [154]:
transforms(np.array(data[0]['image']))

tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 0.3922, 0.3922, 0.3922],
         [0.3922, 0.3922, 0.3686,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 0.7843, 0.0000, 0.6549]]])

In [84]:
def transform_func(examples):
    examples["pixel_values"] = [transforms(np.array(image)) for image in examples["image"]]
    return examples

In [85]:
train_data1 = train_data.map(transform_func, remove_columns=["image"], batched=True)
val_data1 = val_data.map(transform_func, remove_columns=["image"], batched=True)
test_data1 = test_data.map(transform_func, remove_columns=["image"], batched=True)

In [86]:
data_instance = next(iter(train_data1))

In [87]:
data_instance['pixel_values'].shape

torch.Size([1, 224, 224])

In [173]:
train_dataloader = DataLoader(train_data1, batch_size=8)
val_dataloader = DataLoader(val_data1, batch_size=10)

In [109]:
from tqdm import tqdm

In [131]:
i=0
pbar = tqdm(train_dataloader, desc='Training', total=320000/16)
for batch in pbar:
  # print(i)
  i=i+1
  pbar.set_postfix({'batch':pbar.n})
  

Training:   0%|          | 0/20000.0 [00:04<?, ?it/s]


KeyboardInterrupt: ignored

In [71]:
class VectorQuantizer(nn.Module):

  def __init__(self, num_embedding=8192, embedding_dim=512, commitment_cost=0.2):
    super(VectorQuantizer, self).__init__()

    self._num_embeddings = num_embedding
    self._embedding_dim = embedding_dim
    self._embedding = nn.Embedding(num_embedding, embedding_dim)
    self._embedding.weight.data.uniform_(-1.0/num_embedding, 1.0/num_embedding)
    self._commitment_cost = commitment_cost

  def forward(self, inputs):

    B, C, H, W = inputs.shape

    assert C == self._embedding_dim, "Number of channels from encoder output not same as embedding dimension"

    inputs = inputs.permute(0, 2, 3, 1).contiguous()
    input_shape = inputs.shape

    # Flatten input
    flat_input = inputs.view(-1, self._embedding_dim)
    
    # Calculate distances
    distances = (torch.sum(flat_input**2, dim=1, keepdim=True) 
                + torch.sum(self._embedding.weight**2, dim=1)
                - 2 * torch.matmul(flat_input, self._embedding.weight.t()))
        
    # Encoding
    encoding_indices = torch.argmin(distances, dim=1).unsqueeze(1)
    encodings = torch.zeros(encoding_indices.shape[0], self._num_embeddings, device=inputs.device)
    encodings.scatter_(1, encoding_indices, 1)
    
    # Quantize and unflatten
    quantized = torch.matmul(encodings, self._embedding.weight).view(input_shape)

    # Loss
    e_latent_loss = F.mse_loss(quantized.detach(), inputs)
    q_latent_loss = F.mse_loss(quantized, inputs.detach())
    loss = q_latent_loss + self._commitment_cost * e_latent_loss
    
    quantized = inputs + (quantized - inputs).detach()
    avg_probs = torch.mean(encodings, dim=0)
    perplexity = torch.exp(-torch.sum(avg_probs * torch.log(avg_probs + 1e-10)))
    
    # convert quantized from BHWC -> BCHW
    return loss, quantized.permute(0, 3, 1, 2).contiguous(), perplexity, encodings

In [169]:
x = batch['pixel_values']

In [11]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self._block = nn.Sequential(
            nn.ReLU(True),
            nn.Conv2d(in_channels=in_channels,
                      out_channels=in_channels,
                      kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(in_channels=in_channels,
                      out_channels=in_channels,
                      kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(in_channels=in_channels,
                      out_channels=in_channels,
                      kernel_size=1, stride=1, bias=False)
        )
    
    def forward(self, x):
        return x + self._block(x)

In [12]:
class EncoderLayer(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(EncoderLayer, self).__init__()
    
    self.conv_layer = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                                kernel_size=3, stride=2, padding=1, bias=False)
    self.residual_block = ResidualBlock(out_channels)

  def forward(self, input):
    x = self.conv_layer(input)
    x = self.residual_block(x)
    return x

In [34]:
class Encoder(nn.Module):
    def __init__(self, in_channels=1, out_channels=384):
        super(Encoder, self).__init__()
        assert in_channels == 1, 'Number of Input channels not equal to 1'
        self.layers = nn.ModuleList()
        self.layers.append(EncoderLayer(1, 128))
        self.layers.append(EncoderLayer(128, 256))
        self.layers.append(EncoderLayer(256, 384))

    def forward(self, x):
        for layer in self.layers:
          x = layer(x)
        return x

In [30]:
class DecoderLayer(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DecoderLayer, self).__init__()
    
    self.conv_layer = nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, 
                                kernel_size=4, stride=2, padding=1, bias=False)
    self.residual_block = ResidualBlock(out_channels)

  def forward(self, input):
    x = self.conv_layer(input)
    x = self.residual_block(x)
    return x

In [31]:
class Decoder(nn.Module):
  def __init__(self, in_channels=512, out_channels=1):
    super(Decoder, self).__init__()
    self.layers = nn.ModuleList()
    self.layers.append(DecoderLayer(in_channels, in_channels//2))
    self.layers.append(DecoderLayer(in_channels//2, in_channels//4))
    self.layers.append(DecoderLayer(in_channels//4, 1))

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

In [72]:
class Model(nn.Module):
  def __init__(self, enc_in_channels=1, enc_out_channels=384, embedding_dim=512,
               num_embedding=8192, commitment_cost=0.2):
    super(Model, self).__init__()
    self.encoder = Encoder(enc_in_channels, enc_out_channels)
    self.decoder = Decoder(embedding_dim, enc_in_channels)
    self.vqlayer = VectorQuantizer(num_embedding, embedding_dim, commitment_cost)
    self.proj_to_embedding = nn.Conv2d(enc_out_channels, embedding_dim, kernel_size=1, stride=1)

  def forward(self, x):
    B,C,H,W = x.shape

    x = self.encoder(x)
    x = self.proj_to_embedding(x)
    loss, q, perplexity, encodings = self.vqlayer(x)
    x_recon = self.decoder(q)
    return loss, x_recon, perplexity

In [179]:
def train(model, optimizer, history=None):
  model.to(device)
  model.train()
  # print('model device', next(model.parameters()).is_cuda)
  d_model = None

  if history:
    vq_vae_model_history = history
  else:
    vq_vae_model_history = dict(loss_lst=[], recon_loss_lst=[], perplexity_lst=[])

  i = 0
  pbar = tqdm(train_dataloader, desc='Training', total=320000/16)
  for batch in pbar:
    batch = batch['pixel_values']
    # print('device 1', batch.device)
    batch = batch.to(device)
    # print('device 2', batch.device)

    optimizer.zero_grad()
    loss, x_recon, perplexity = model(batch)

    recon_loss = F.mse_loss(batch, x_recon)
    loss_final = loss+recon_loss-perplexity
    loss_final.backward()

    optimizer.step()

    vq_vae_model_history['loss_lst'].append(loss.item())
    vq_vae_model_history['recon_loss_lst'].append(recon_loss.item())
    vq_vae_model_history['perplexity_lst'].append(perplexity.item())

    pbar.set_postfix({'loss':loss_final.item()})

    if pbar.n>500*i:
      i=i+1
      MODEL_PATH = 'vq-vae_doc_model.pth'
      torch.save(model, MODEL_PATH)

      with open('vq_vae_model_history', 'wb') as f:
        pickle.dump(vq_vae_model_history, f)

      if d_model:
        d_model.Delete()
        d_history.Delete()

      # Create & upload a file.
      d_model = gdrive.CreateFile({'vq-vae_doc_model.pth': 'vq-vae_doc_model.pth'})
      d_model.SetContentFile('vq-vae_doc_model.pth')
      d_model.Upload()
      print('\nUploaded file with ID {}'.format(d_model.get('id')))

      
      # Create & upload a file.
      d_history = gdrive.CreateFile({'vq_vae_model_history': 'vq_vae_model_history'})
      d_history.SetContentFile('vq_vae_model_history')
      d_history.Upload()
      print('\nUploaded file with ID {}'.format(d_history.get('id')))

In [156]:
model = Model()

In [136]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [180]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
f'Number of parameters: {round(pytorch_total_params/10_00_000, 3)}M'

'Number of parameters: 14.11M'

In [ ]:
train(model, optimizer)

Training:   0%|          | 1/20000.0 [00:11<63:55:29, 11.51s/it, loss=-2.24]


Uploaded file with ID 1_9_BvAlT-bJppeJNMYlQuhkf-Eb43kMr


Training:   0%|          | 2/20000.0 [00:14<37:02:02,  6.67s/it, loss=-2.24]


Uploaded file with ID 1ysVcwM8wh05hcErWmMlI_nGcKfe0s23K


Training:   3%|▎         | 501/20000.0 [02:58<14:41:36,  2.71s/it, loss=-1.55]

...

Training:  58%|█████▊    | 11501/20000.0 [1:05:37<6:41:30,  2.83s/it, loss=-1.27]


Uploaded file with ID 1UZSt1zAZxVM2Dw0wXy4stj2XLTi3DiJb


Training:  58%|█████▊    | 11502/20000.0 [1:05:42<8:06:22,  3.43s/it, loss=-1.27]


Uploaded file with ID 17hRxSEP4NyOLHbqKTMD69l8JAtmJkELj


Training:  59%|█████▉    | 11875/20000.0 [1:07:37<36:01,  3.76it/s, loss=-1.44]

In [ ]:
model